# B1 - Quick Intro to Chebyshev Polynomials

In [ ]:
using Pkg; Pkg.activate(".")
using LaTeXStrings, WGLMakie, LinearAlgebra

### Runge's Phenomenon

We consider the function $f : [-1, 1] \to \mathbb{R}$, 
$$
   f(x) = \frac{1}{1 + 25 x^2}
$$
Note that $f$ is analytic on $[-1,1]$, hence from our work on trigonometric approximation we expect excellent approximation properties. We choose a uniform grid, 
$$
  x_j = -1 + 2j/N, \qquad j = 0, \dots, N
$$
and interpolate $f$ at those grid points. 

In [ ]:
f = x -> 1/(1+25*x^2)
NN1 = [5, 8, 10]
NN2 =  5:5:30

# do not do this!!! - we will learn later how to do 
# polynomial interpolation in a numerically stable way.
function naive_poly_fit(f, N)
   X = range(-1, 1, length=N+1)
   A = [ X[m+1]^n for m = 0:N, n = 0:N ]
   return A \ f.(X)
end

# don't do this either, this is just a quick naive code 
# suitable for the current experiment. 
naive_poly_eval(x, c) = sum( c[n] * x^(n-1) for n = 1:length(c) )

# first plot 
xp = range(-1, 1, length=300)
fig1 = Figure(size = (600, 300), title = "Witch of Agnesi")
ax11 = Axis(fig1[1,1], xlabel = L"x")
lines!(ax11, xp, f.(xp); linewidth=4, label = "exact")
for (iN, N) in enumerate(NN1)
   xi = [(-1 + 2*m/N) for m = 0:N]
   c = naive_poly_fit(f, N)
   lines!(ax11, xp, naive_poly_eval.(xp, Ref(c)), linewidth=2,label = L"p_{%$(N)}")
#    plot!(P1, xi, f.(xi), lw=0, c = iN+1, m = :o, ms=3, label = "")
end 
axislegend(ax11)


# second plot 
xerr = range(-1, 1, length=3_000)
err = [ norm( f.(xerr) - naive_poly_eval.(xerr, Ref(naive_poly_fit(f, N))), Inf )
        for N in NN2 ]
ax12 = Axis(fig1[1,2], yscale = log10, xlabel = L"N")
lines!(NN2, err, linewidth = 3, label = L"\Vert f - I_N f \Vert")
axislegend(ax12)

fig1

### The Joukowsky Map 

Everything we developed for trigonometric polynomials can still be transferred to algebraic polynomials, but in a far less obvious way.

Start with a periodic function that is analytic on some strip, indicated by the blue lines. 

In [ ]:
tt = range(0, 2*pi, length=300)
aa = [0.1:0.1:0.5; -0.5:0.1:-0.1]

fig2 = Figure(size=(650, 300))

ax21 = Axis(fig2[1,1], ylabel = "θ")
lines!(tt, 0*tt, color=:red, linewidth=3, label = "")
for a in aa
    lines!(tt, imag.(tt .+ im*a), linewidth=1, color=:blue, label = "")
end

ax22 = Axis(fig2[1,2])
f_fun = θ -> 1 / (1 + 3^2 * cos(θ)^2)
plt2 = lines!(tt, f_fun.(tt), linewidth=3, color=:black, label = "f(θ)")

fig2

Now transform this strip + PBC to the complex plane, with the real axis mapping to the unit circle (torus). The transformation that maps the variable $\theta \in \mathbb{R}$ to the unit circle is simply 
$$
 \theta \mapsto z = e^{i \theta}
$$


In [ ]:
fig3 = Figure(size=(700, 300))
ax31 = Axis(fig3[1,1], title = "z = exp(i θ)")
lines!(ax31, cos.(tt), sin.(tt), linewidth=2, color=:red )
for a in aa
    u = exp.(im * (tt .+ im * a))
    lines!(ax31, real.(u), imag.(u), linewidth=1, color=:blue, label = "")
end

zz = exp.(im * tt); rr = real.(zz); ii = imag.(zz)
ax32 = Axis3(fig3[1, 2])
lines!(rr, ii, 0*rr, color = :red, linewidth=3, label = "")
lines!(rr, ii, f_fun.(tt), color = :black, linewidth=3, label = "")

fig3

Now notice that the reflection symmetry (kind of by choice) of the function $g(z) = f(\theta)$, i.e. $g(z) = g(z^*)$. Suppose, that we define a new function 
$$
   h(\cos\theta) = f(\theta) = g(\cos\theta + i \sin\theta),
$$
then $h : [-1, 1] \to \mathbb{R}$. And in fact $h(x)$ is the classical witch of Agnesi: 
$$
    h(x) = \frac{1}{1+c^2 x^2}.
$$
The transform $x = \cos\theta$ extends to the complex plane via 
$$ \begin{aligned}
    z = \frac{e^{i \theta} + e^{- i \theta}}{2},
\end{aligned}$$
where $\theta$ is now complex.

Specifically we are interested in how a strip $\Omega_\alpha$ transforms under these transformations. We can see this in the next figure.

In [ ]:
fig4 = Figure(size = (700, 300))
ax41 = Axis(fig4[1,1]; title = "z = (u+u⁻¹)/2", limits=(-1.4, 1.4, -0.8, 0.8))
lines!(ax41, cos.(tt), 0*tt, linewidth=2, color=:red, label = "")
for a in aa
    u = exp.(im * (tt .+ im * a)); z = (u .+ 1 ./u)/2
    lines!(ax41, real.(z), imag.(z), linewidth=1, color=:blue, label = "")
end

ax42 = Axis(fig4[1,2],  xlabel = "x = cosθ")
lines!(ax42, cos.(tt), f_fun.(tt), linewidth=3, color = :black,
            label = "f(θ) = h(x)",)

fig4

We can now ask how everything we know about Trigonoemtric polynomial approximation transforms to chebyshev polynomials? 

- Equispaced nodes $j \pi/N$ become Chebyshev nodes: $x_j = \cos(j\pi/N)$
- The trigonometric basis $e^{i k \theta}$ becomes the Chebyshev basis: $T_k(\cos\theta) = \cos(k \theta)$. (Note that reflection symmetry is needed here and this gives symmetry in the coefficients and therefore requires only the cosine series.)
- The Fourier series becomes the Chebyshev series
- The strip $\Omega_\alpha$ becomes the Bernstein ellipse $E_\rho$ (with related parameters $\rho, \alpha$)
- The FFT can be used to transform between nodal values at Chebyshev nodes and Chebyshev coefficients.

More details in class notes, or see 
- Nick Trefethen, Approximation Theory and Approximation Practice

Equipped with this information, we can have a first stab at fixing the Runge phenomenon.

In [ ]:
function chebbasis(x, N)
   T = zeros(N+1)
   T[1] = 1 
   T[2] = x 
   for n = 2:N
      T[n+1] = 2 * x * T[n] - T[n-1] 
   end 
   return T 
end

chebnodes(N) = [ cos( π * n / N ) for n = N:-1:0 ]

# This implementation does not yet use the FFT!!!
function chebinterp(f, N)
   X = chebnodes(N)
   A = zeros(N+1, N+1)
   for (ix, x) in enumerate(X)
      A[ix, :] .= chebbasis(x, N)
   end
   return A \ f.(X)
end

chebeval(x, c) = dot(c, chebbasis(x, length(c)-1))

In [ ]:
xp = range(-1, 1, length=300)

fig5 = Figure(size = (600, 300))

ax51 = Axis(fig5[1,1], xlabel = L"x", title = "Witch of Agnesi")
lines!(ax51, xp, f.(xp); linewidth=4, label = "exact")
for (iN, N) in enumerate(NN1)
   xi = [(-1 + 2*m/N) for m = 0:N]
   c = chebinterp(f, N)
   lines!(ax51, xp, chebeval.(xp, Ref(c)), label = L"p_{%$(N)}")
#    plot!(P1, xi, f.(xi), lw=0, c = iN+1, m = :o, ms=3, label = "")
end 
axislegend(ax51)

# second plot 
xerr = range(-1, 1, length=3_000)
err = [ norm( f.(xerr) - chebeval.(xerr, Ref(chebinterp(f, N))), Inf )
      for N in NN2 ]
ax52 = Axis(fig5[1,2], yscale = log10, xlabel = L"N")
scatterlines!(ax52, NN2, err, label = L"\Vert f - I_N f \Vert")
lines!(ax52, NN2[3:end], 4*(1.23).^(-NN2[3:end]), color=:black, linestyle=:dash, label = L"\rho^{-N}")
axislegend(ax52)

fig5

---

## The Fast Chebyshev Transform

Because of the intimate connection between Chebyshev polynomials and trigonometric polynomials we can use the FFT to implement the interpolation operator.

In [ ]:
using FFTW 

function fct(A::AbstractVector)
    N = length(A)
    F = real.(ifft([A[1:N]; A[N-1:-1:2]]))
   return [[F[1]]; 2*F[2:(N-1)]; [F[N]]]
end

"""
Fast and stable implementation based on the FFT. This uses 
the connection between Chebyshev and trigonometric interpolation.
But this transform needs the reverse chebyshev nodes.
"""
chebinterp(f, N) = fct(f.(reverse(chebnodes(N))))

"""
Evaluate a polynomial with coefficients F̃ in the Chebyshev basis. 
This avoids storing the basis and is significantly faster.
"""
function chebeval(x, F̃) 
    T0 = one(x); T1 = x 
    p = F̃[1] * T0 + F̃[2] * T1 
    for n = 3:length(F̃)
        T0, T1 = T1, 2*x*T1 - T0 
        p += F̃[n] * T1 
    end 
    return p 
end 


In [ ]:
xp = range(-1, 1, length=300)

fig6 = Figure(size=(600, 300))
ax61 = Axis(fig6[1,1], xlabel = L"x", title = "Witch of Agnesi")
lines!(ax61, xp, f.(xp); linewidth=4, label = "exact")
for (iN, N) in enumerate(NN1)
   xi = [(-1 + 2*m/N) for m = 0:N]
   c = chebinterp(f, N)
   lines!(ax61, xp, chebeval.(xp, Ref(c)), linewidth=2,label = L"p_{%$(N)}")
#    plot!(P1, xi, f.(xi), lw=0, c = iN+1, m = :o, ms=3, label = "")
end 
axislegend(ax61)

# second plot 
xerr = range(-1, 1, length=3_000)
err = [ norm( f.(xerr) - chebeval.(xerr, Ref(chebinterp(f, N))), Inf )
      for N in NN2 ]
ax62 = Axis(fig6[1,2], yscale = log10, xlabel = L"N")
scatterlines!(ax62, NN2, err, linewidth = 3, label = L"\Vert f - I_N f \Vert")
lines!(ax62, NN2[3:end], 4*(1.23).^(-NN2[3:end]), color=:black, linestyle=:dash, label = L"\rho^{-N}")
axislegend(ax62)

fig6


Let's take this to the extreme, but constructing a much harder problem. With FCT we can easily scale to very high polynomial degrees : this is both very fast and numerically perfectly stable. 

In [ ]:
xp = range(-1, 1, length=500)
f = x -> 1 / (1 + 2000 * x^2)
NN = 16:16:1_600


fig6 = Figure(size=(600, 300))
ax61 = Axis(fig6[1,1], xlabel = L"x", title = "Witch of Agnesi")
lines!(ax61, xp, f.(xp); linewidth=4, label = "")

# second plot 
xerr = range(-1, 1, length=3_000)
err = [ norm( f.(xerr) - chebeval.(xerr, Ref(chebinterp(f, N))), Inf )
      for N in NN ]
ax62 = Axis(fig6[1,2], yscale = log10, xlabel = L"N")
lines!(ax62, NN, err, linewidth = 3, label = L"\Vert f - I_N f \Vert")
tt = [250, 750]
lines!(ax62, tt, 4*(1+1/sqrt(2000)).^(-tt), color=:black, linestyle=:dash, label = L"\rho^{-N}")
axislegend(ax62)

fig6
